In [84]:
import numpy as np
import os

FILE = './bitfile.bin'
INT32_SIZE = 4

In [85]:
def print_result(_sum, min_value, max_value):
    print(f"Сумма = {_sum};\nМинимальное значение = {min_value};\nМаксимальное значение: {max_value}.")


1. Простое последовательное чтение 

Чтение с numpy

In [86]:
%%time
def read_fromfile(filepath = FILE):
    return np.fromfile(filepath, dtype=np.uint32)

numbers = read_fromfile()
print_result(np.sum(numbers), np.min(numbers), np.max(numbers))
print(f"Размер файла: {len(numbers) * 4 / (1024**3)} Гб.")


Сумма = 1152904582664695789;
Минимальное значение = 6;
Максимальное значение: 4294967289.
Размер файла: 2.0 Гб.
CPU times: user 1.35 s, sys: 3.86 s, total: 5.22 s
Wall time: 5.8 s


In [87]:
%%time
def read_frombuffer(filepath = FILE):
    with open(filepath, 'rb') as f:
        return np.frombuffer(f.read(), dtype=np.uint32)

numbers = read_frombuffer()
print_result(np.sum(numbers), np.min(numbers), np.max(numbers))


Сумма = 1152904582664695789;
Минимальное значение = 6;
Максимальное значение: 4294967289.
CPU times: user 1.48 s, sys: 1.55 s, total: 3.03 s
Wall time: 3.7 s


Чтение без numpy

In [88]:
%%time
def bin_read(filepath = FILE):
    num_sum = 0
    with open(filepath, 'rb') as f:
        int32_bytes = f.read(INT32_SIZE)
        min_value = int.from_bytes(int32_bytes, byteorder='big', signed=False)
        max_value = min_value
        while int32_bytes:
            new_value = int.from_bytes(int32_bytes, byteorder='big', signed=False)
            num_sum += new_value
            min_value = min(min_value, new_value)
            max_value = max(max_value, new_value)
            int32_bytes = f.read(INT32_SIZE)

    return num_sum, min_value, max_value

num_sum, min_value, max_value = bin_read()
print_result(num_sum, min_value, max_value)


Сумма = 1152888645987318649;
Минимальное значение = 1;
Максимальное значение: 4294967285.
CPU times: user 4min 10s, sys: 1.49 s, total: 4min 12s
Wall time: 4min 13s


2. Многопоточная + memory-mapped files

In [126]:
import threading
from multiprocessing import Process, Manager
import mmap

lock = threading.Lock()
global thread_results


In [150]:
def mmap_params_with_index(file, max_threads):
    file_size = os.stat(FILE).st_size
    batch_size = mmap.ALLOCATIONGRANULARITY
    batchs_count = -(-file_size // batch_size) # Округляем в большую сторону
    batch_per_thread = -(-batchs_count // max_threads)
    thread_size = batch_size*batch_per_thread
    for i in range(max_threads):
        offset = i*thread_size
        length = min(file_size - offset, thread_size)

        yield i, {'fileno': file.fileno(),
                  'length': length,
                  'offset': offset,
                  'access': mmap.ACCESS_READ}

def thread_target(mmap_params):
    global thread_results
    thread_results = [0, 2*32 + 1, -1]
    num_sum = 0
    with mmap.mmap(**mmap_params) as mm:
        int32_bytes = mm.read(INT32_SIZE)
        min_value = int.from_bytes(int32_bytes, byteorder='big', signed=False)
        max_value = min_value
        while int32_bytes:
            new_value = int.from_bytes(int32_bytes, byteorder='big', signed=False)
            num_sum += new_value
            min_value = min(min_value, new_value)
            max_value = max(max_value, new_value)
            int32_bytes = mm.read(INT32_SIZE)
            
    with lock:
        thread_results[0] += num_sum
        thread_results[1] = min(min_value, thread_results[1])
        thread_results[2] = max(max_value, thread_results[2])

def np_thread_target(mmap_params):
    global thread_results
    thread_results = [0, 2*32 + 1, -1]
    with mmap.mmap(**mmap_params) as mm:
        numbers = np.frombuffer(mm.read(), dtype=np.uint32)

    with lock:
        num_sum, min_value, max_value = [np.sum(numbers), np.min(numbers), np.max(numbers)]
        thread_results[0] += num_sum
        thread_results[1] = min(min_value, thread_results[1])
        thread_results[2] = max(max_value, thread_results[2])



Потоки с обычным чтением

In [124]:
def mmap_with_threads_read(filepath = FILE):
    global thread_results

    num_threads = 8
    threads = []   
    with open(filepath, 'rb') as f:    
        for _, mmap_params in mmap_params_with_index(f, num_threads):
            thread = threading.Thread(target=thread_target, args=[mmap_params])
            threads.append(thread)

        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()

    return thread_results


In [133]:
%%time
result = mmap_with_threads_read()
print_result(result[0], result[1], result[2])

Сумма = 1152888645987318649;
Минимальное значение = 1;
Максимальное значение: 4294967285.
CPU times: user 4min 29s, sys: 2.35 s, total: 4min 31s
Wall time: 4min 26s


Потоки с использованием numpy

In [134]:
def np_mmap_with_threads_read(filepath = FILE, num_threads = 8):
    global thread_results

    threads = []   
    with open(filepath, 'rb') as f:    
        for _, mmap_params in mmap_params_with_index(f, num_threads):
            thread = threading.Thread(target=np_thread_target, args=(mmap_params,))
            threads.append(thread)

        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()

    return thread_results


8 потоков

In [135]:
%%time
result = np_mmap_with_threads_read()
print_result(result[0], result[1], result[2])


Сумма = 1.1529045826646958e+18;
Минимальное значение = 6;
Максимальное значение: 4294967289.
CPU times: user 1.85 s, sys: 1e+03 ms, total: 2.85 s
Wall time: 3.04 s


4 потока

In [137]:
%%time
result = np_mmap_with_threads_read(num_threads = 4)
print_result(result[0], result[1], result[2])


Сумма = 1.1529045826646958e+18;
Минимальное значение = 6;
Максимальное значение: 4294967289.
CPU times: user 2.02 s, sys: 801 ms, total: 2.82 s
Wall time: 2.37 s


Чтение через процессы

In [142]:
def proccess_target(index, mmap_params, return_list):
    num_sum = 0
    with mmap.mmap(**mmap_params) as mm:
        int32_bytes = mm.read(INT32_SIZE)
        min_value = int.from_bytes(int32_bytes, byteorder='big', signed=False)
        max_value = min_value
        while int32_bytes:
            new_value = int.from_bytes(int32_bytes, byteorder='big', signed=False)
            num_sum += new_value
            min_value = min(min_value, new_value)
            max_value = max(max_value, new_value)
            int32_bytes = mm.read(INT32_SIZE)
            
    return_list[index] = num_sum, min_value, max_value

def np_proccess_target(index, mmap_params, return_list):
    with mmap.mmap(**mmap_params) as mm:
        numbers = np.frombuffer(mm.read(), dtype=np.uint32)

    return_list[index] = np.sum(numbers), np.min(numbers), np.max(numbers)


In [139]:
def mmap_with_process_read(filepath = FILE, num_threads = 8):
    result = Manager().dict()
    threads = []   
    with open(filepath, 'rb') as f:    
        for i, mmap_params in mmap_params_with_index(f, num_threads):
            thread = Process(target=proccess_target, args=(i, mmap_params, result))
            threads.append(thread)

        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()

    return result


8 потоков

In [140]:
%%time
result = mmap_with_process_read()
print_result(np.sum(list(map(lambda v: v[0], result.values()))),\
             np.min(list(map(lambda v: v[1], result.values()))),\
             np.max(list(map(lambda v: v[2], result.values()))))

Сумма = 1152888645987318649;
Минимальное значение = 1;
Максимальное значение: 4294967285.
CPU times: user 2.8 ms, sys: 352 ms, total: 355 ms
Wall time: 1min 54s


4 потока

In [141]:
%%time
result = mmap_with_process_read(num_threads = 4)
print_result(np.sum(list(map(lambda v: v[0], result.values()))),\
             np.min(list(map(lambda v: v[1], result.values()))),\
             np.max(list(map(lambda v: v[2], result.values()))))

Сумма = 1152888645987318649;
Минимальное значение = 1;
Максимальное значение: 4294967285.
CPU times: user 6.48 ms, sys: 231 ms, total: 237 ms
Wall time: 1min 58s


Чтение через процессы с numpy

In [143]:
def np_mmap_with_proccess_read(filepath = FILE, num_threads = 8):
    result = Manager().dict()
    threads = []   
    with open(filepath, 'rb') as f:    
        for i, mmap_params in mmap_params_with_index(f, num_threads):
            thread = Process(target=np_proccess_target, args=(i, mmap_params, result))
            threads.append(thread)

        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()

    return result


8 процессов

In [144]:
%%time
result = np_mmap_with_proccess_read()
print_result(np.sum(list(map(lambda v: v[0], result.values()))),\
             np.min(list(map(lambda v: v[1], result.values()))),\
             np.max(list(map(lambda v: v[2], result.values()))))

Сумма = 1152904582664695789;
Минимальное значение = 6;
Максимальное значение: 4294967289.
CPU times: user 2.37 ms, sys: 648 ms, total: 650 ms
Wall time: 1.72 s


4 процесса

In [151]:
%%time
result = np_mmap_with_proccess_read(num_threads = 4)
print_result(np.sum(list(map(lambda v: v[0], result.values()))),\
             np.min(list(map(lambda v: v[1], result.values()))),\
             np.max(list(map(lambda v: v[2], result.values()))))

Сумма = 1152904582664695789;
Минимальное значение = 6;
Максимальное значение: 4294967289.
CPU times: user 6.65 ms, sys: 207 ms, total: 213 ms
Wall time: 1.54 s


Заключение:
- Чтение с numpy быстрее чем простое чтение;
- Чтение с отдельными процессами быстрее чем через потоки, потоки быстрее чем последовательное чтение.